# Load Data

In [38]:
import pandas as pd 

# columns correspond to unitigs so we must transpose this table
def load_unitig_data(path) -> pd.DataFrame:
    sr = pd.read_csv(path, sep=' ')
    sr.set_index('pattern_id',inplace=True)
    return sr.T[1:]

azm_sr = load_unitig_data("azm_sr_gwas_filtered_unitigs.Rtab")
cfx_sr = load_unitig_data("cfx_sr_gwas_filtered_unitigs.Rtab")
cip_sr = load_unitig_data("cip_sr_gwas_filtered_unitigs.Rtab")

metadata = pd.read_csv('metadata.csv')
metadata.set_index('Sample_ID',inplace=True)


In [39]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            3536 non-null   float64
 1   Country         3785 non-null   object 
 2   Continent       3785 non-null   object 
 3   Beta.lactamase  1927 non-null   object 
 4   Azithromycin    3480 non-null   object 
 5   Ciprofloxacin   3129 non-null   object 
 6   Ceftriaxone     3436 non-null   object 
 7   Cefixime        3405 non-null   object 
 8   Tetracycline    1472 non-null   object 
 9   Penicillin      1465 non-null   object 
 10  NG_MAST         3779 non-null   object 
 11  Group           3786 non-null   int64  
 12  azm_mic         3478 non-null   float64
 13  cip_mic         3088 non-null   float64
 14  cro_mic         3434 non-null   float64
 15  cfx_mic         3401 non-null   float64
 16  tet_mic         1472 non-null   float64
 17  pen_mic         1465 no

In [40]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


# Null/NA Cleaning in Labels

1. Remove rows with NaN in the labels we are trying to predict: 'azm_sr','cfx_sr', 'cip_sr'

---

Note to Jacob:
If I remove all rows with nulls, then we drop down to ~1k entries. Not ideal. I will remove only the nulls in the target labels. By doing that I was able to preserve ~2800 entries.

Additionally, I can technically replace the nulls in the feature set with averages whether they are continuous or discrete, but since we are going to be building some kind of predictive model later, it would be bad practice to run column averages in df.fillna() before we split our dataset into training and test sets.

Therefore, I am going to split the dataset into a training and test set first.

What do you think?

In [41]:
metadata.dropna(axis=0, how='any', inplace=True, subset=['azm_sr', 'cfx_sr', 'cip_sr'])

In [42]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
SRR1661154,1996.0,Canada,America,NaN,0.25,0.5,0.125,0.125,2,2,...,-3.000000,-3.000000,1.0,1.000000,0.0,0.0,0.0,0.0,1.0,1.0
SRR1661156,1997.0,Canada,America,NaN,0.25,16,0.125,0.063,2,2,...,-3.000000,-3.988504,1.0,1.000000,0.0,1.0,0.0,0.0,1.0,1.0
SRR1661157,1998.0,Canada,America,NaN,0.125,0.016,0.125,0.063,>=128,4,...,-3.000000,-3.988504,8.0,2.000000,0.0,0.0,0.0,0.0,1.0,1.0
SRR1661158,2000.0,Canada,America,NaN,0.25,0.5,0.125,0.25,4,4,...,-3.000000,-2.000000,2.0,2.000000,0.0,0.0,0.0,0.0,1.0,1.0
SRR1661159,2001.0,Canada,America,NaN,0.063,0.063,0.00025,0.001,0.5,0.008,...,-11.965784,-9.965784,-1.0,-6.965784,0.0,0.0,0.0,0.0,0.0,0.0


# Removing Un-needed Columns
1. Removing 'Year', and unimportant labels 'cro_sr', 'tet_sr', 'pen_sr'

In [44]:
useless_columns = ['Year', 'cro_sr', 'tet_sr', 'pen_sr']

metadata.drop(labels=useless_columns, axis=1, inplace=True)

In [45]:
metadata.head(5)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
SRR1661154,Canada,America,NaN,0.25,0.5,0.125,0.125,2,2,10443,...,2.000,-2.000000,-1.000000,-3.000000,-3.000000,1.0,1.000000,0.0,0.0,0.0
SRR1661156,Canada,America,NaN,0.25,16,0.125,0.063,2,2,10476,...,2.000,-2.000000,4.000000,-3.000000,-3.988504,1.0,1.000000,0.0,1.0,0.0
SRR1661157,Canada,America,NaN,0.125,0.016,0.125,0.063,>=128,4,10469,...,4.000,-3.000000,-5.965784,-3.000000,-3.988504,8.0,2.000000,0.0,0.0,0.0
SRR1661158,Canada,America,NaN,0.25,0.5,0.125,0.25,4,4,4138,...,4.000,-2.000000,-1.000000,-3.000000,-2.000000,2.0,2.000000,0.0,0.0,0.0
SRR1661159,Canada,America,NaN,0.063,0.063,0.00025,0.001,0.5,0.008,8,...,0.008,-3.988504,-3.988504,-11.965784,-9.965784,-1.0,-6.965784,0.0,0.0,0.0


# Cleaning non-numeric entries in numeric fields to NaN.

1. Turn Non Numeric Entries in Numeric Columns into NaN
2. Cast all numeric rows into float32
---


Notes: Turning them into NaN for now. Will engineer values for all NaNs after train and test splits are made

In [47]:
numeric_columns = [
'Azithromycin',
'Ciprofloxacin',
'Ceftriaxone',
'Cefixime',
'Tetracycline',
'Penicillin',
'NG_MAST',
'Group',
'azm_mic',
'cip_mic',
'cro_mic',
'cfx_mic',
'tet_mic',
'pen_mic',
'log2_azm_mic',
'log2_cip_mic',
'log2_cro_mic',
'log2_cfx_mic',
'log2_tet_mic',
'log2_pen_mic',
'azm_sr',
'cip_sr',
'cfx_sr',
]

for column in numeric_columns:
    metadata[column] = pd.to_numeric(metadata[column], errors='coerce', downcast="float") #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

In [48]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2814 entries, SRR1661154 to SRR5827370
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         2813 non-null   object 
 1   Continent       2813 non-null   object 
 2   Beta.lactamase  1865 non-null   object 
 3   Azithromycin    2812 non-null   float32
 4   Ciprofloxacin   2796 non-null   float32
 5   Ceftriaxone     2767 non-null   float32
 6   Cefixime        2795 non-null   float32
 7   Tetracycline    1264 non-null   float32
 8   Penicillin      1261 non-null   float32
 9   NG_MAST         2336 non-null   float32
 10  Group           2814 non-null   float32
 11  azm_mic         2814 non-null   float32
 12  cip_mic         2814 non-null   float32
 13  cro_mic         2769 non-null   float32
 14  cfx_mic         2814 non-null   float32
 15  tet_mic         1266 non-null   float32
 16  pen_mic         1264 non-null   float32
 17  log2_azm_mic    2814 no

In [49]:
metadata.head(10)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
SRR1661154,Canada,America,NaN,0.250,0.500,0.12500,0.125,2.000,2.000,10443.0,...,2.000,-2.000000,-1.000000,-3.000000,-3.000000,1.0,1.000000,0.0,0.0,0.0
SRR1661156,Canada,America,NaN,0.250,16.000,0.12500,0.063,2.000,2.000,10476.0,...,2.000,-2.000000,4.000000,-3.000000,-3.988504,1.0,1.000000,0.0,1.0,0.0
SRR1661157,Canada,America,NaN,0.125,0.016,0.12500,0.063,NaN,4.000,10469.0,...,4.000,-3.000000,-5.965784,-3.000000,-3.988504,8.0,2.000000,0.0,0.0,0.0
SRR1661158,Canada,America,NaN,0.250,0.500,0.12500,0.250,4.000,4.000,4138.0,...,4.000,-2.000000,-1.000000,-3.000000,-2.000000,2.0,2.000000,0.0,0.0,0.0
SRR1661159,Canada,America,NaN,0.063,0.063,0.00025,0.001,0.500,0.008,8.0,...,0.008,-3.988504,-3.988504,-11.965784,-9.965784,-1.0,-6.965784,0.0,0.0,0.0
SRR1661160,Canada,America,NaN,0.125,0.004,0.00200,0.004,1.000,0.063,NaN,...,0.063,-3.000000,-7.965784,-8.965784,-7.965784,0.0,-3.988504,0.0,0.0,0.0
SRR1661161,Canada,America,NaN,0.500,16.000,0.12500,0.250,4.000,4.000,4383.0,...,4.000,-1.000000,4.000000,-3.000000,-2.000000,2.0,2.000000,0.0,1.0,0.0
SRR1661162,Canada,America,NaN,0.063,0.004,0.00400,0.008,0.125,0.063,51.0,...,0.063,-3.988504,-7.965784,-7.965784,-6.965784,-3.0,-3.988504,0.0,0.0,0.0
SRR1661163,Canada,America,NaN,0.500,0.032,0.12500,0.032,4.000,4.000,108.0,...,4.000,-1.000000,-4.965784,-3.000000,-4.965784,2.0,2.000000,0.0,0.0,0.0


# Split Dataframe into Train and Test

# Re-engineer NaN features if possible

### Data Stats

In [ ]:
j=[0,0,0]
samples = metadata.index
for sample in samples:
    if metadata['azm_sr'][sample]:
        j[0] +=1
    if metadata['cfx_sr'][sample]:
        j[1] +=1
    if metadata['cip_sr'][sample]:
        j[2] +=1

print(j[0]/len(samples), "% of samples have resistance to azm")
print(j[1]/len(samples), "% of samples have resistance to cfx")
print(j[2]/len(samples), "% of samples have resistance to cip")


In [ ]:
samples = azm_sr.index

# some random unitig from azm_sr

from random import randint
randomUnitig = azm_sr.columns[randint(0,azm_sr.shape[1])]
j = 0
print(azm_sr.shape)
for i in range(azm_sr.shape[0]):
    if azm_sr[randomUnitig][samples[i]]:
        j += 1

print(randomUnitig,"\npresent in", j/azm_sr.shape[0],"% of azm_sr samples (",j,"/",azm_sr.shape[0],')' )